In [6]:
## Dictionary-Based Representation of Vectors
A vector is a function from some domain **D** to a field. I proposed to represent such a function using a Python 
dictionary. It is convenient to define a 
Python class Vec so that an instance ha two fields (also know as intance variables, also known as attributes):

- **f**, the function, representd by a Python dictionary, and
- **D**, the domain of the function, represented by a Python set.

We adopt the convention in which entries with value zero may be omitted from the dictionary **f**. 
This enables sparse vectors to be represented compactly. 
It might seem a bad idea to require that each instance of Vec keep track of the domain. 
For example, as previously pointed, in information retrieval one typically  has many documents,
each including only a very small subset of words; 
it would waste memory to duplicate the entire list of allowed words with each document. 
Fortunately, as we saw previously, Python allows many variables (or instance variables) to point to the same set in memory. 
This, if we are careful, we can ensure that all the vectors representing documents point to the same domain.

The Python code required to define the class Vec is:

SyntaxError: invalid syntax (<ipython-input-6-73f4ef3077a0>, line 2)

In [ ]:
class Vec:
    def __init__(self, labels, function):
        self.D = labels
        self.f = function

In [ ]:
v = Vec({"A", "B", "C"}, {"A": 1})

for d in v.D:
    if d in v.f:
        print(v.f[d])

## Vector Class Methods
### Addition, Scalar Multiplication and Negation of a Vector

In [ ]:
def get_item(v, d):
    if d in v.f:
        return v.f[d]
    else:
        return 0

def set_item(v, d, val):
    if d in v.D:
        v.f[d] = val
    else:
        print("Outside of Domain")

def scalar_mul(v, alpha):
    #Preserves Sparsity
    return Vec(v.D, {d: alpha*value for d, value in v.f.items()})

def add(u, v):
    return Vec({d for d in v.D if d in u.D}, {d: get_item(u, d)+get_item(v, d) for d in v.D if get_item(u, d)+get_item(v, d) != 0})

def neg(v):
    return Vec(v.D, {d: -value for d, value in v.f.items()})

### Testing Methods

In [ ]:
v_scaled = scalar_mul(v, 2)
v_scaled.f

In [ ]:
u = Vec(v.D, {"A": 5, "C": 10})
v_sum = add(u, v)
v_sum.f

In [ ]:
add(v_sum, neg(u)).f

## GF(2) Field - Perfect Secrecy Revised
Recall **Alice** and **Bob** and their need for prefect secrecy. We saw in previously that encrypting a single-bit plaintext consisted of adding that bit to a single-bit key, using **GF(2)** addition. We saw also that, to encrypt a sequence of bits of plaintext, it sufficed to just encrypt each bit with its own bit of key. That process can be expresed more compactly using addition of vectors over **GF(2)**.

**GF(2)** : 
- Addition is module 2. It is equivalent to **XOR** . Ex. 1 + 1 = 0.
- Substraction if identical to addition. The negative of 1 is again 1, and the negative of 0 is again 0.
- Multiplication in GF(2) is just like orginary multiplication of 0 and 1: It is equivalent to the bitwise operator **AND** .

Suppose Alice needs to send a ten-bit plaintext **p** to Bob

In [ ]:
# For example, Alice and Bob agree on the following 10-vectors as a key:
k = [0, 1, 1, 0, 1, 0, 0, 0, 0, 1]

# Alice wants to send this message to Bob:
p = [0, 0, 0, 1, 1, 1, 0, 1, 0, 1]

# She encrypts it by adding p to k:
c = [x ^ y for x, y in zip(k, p)]
c

In [ ]:
# When bob reviesves c, he decrypts it by ading k:
message = [x ^ y for x, y in zip(c, k)]
message

Next we check that the system satisfies perfect secrecy. The argument should be familiar. For each plaintext **p**, the function **k -> k + p** is one-to-one and onto, hence invertible. Since the key **k** is chosen uniformly at random, therefore, the cyphertext **c** is also distributed uniformly.

### Dot Product of a List

In [ ]:
def list_dot(u, v):
    return sum([x*y for (x, y) in zip(u, v)])

## Full Vector Implementation

In [ ]:
def getitem(v,d):
    "Returns the value of entry d in v"
    assert d in v.D
    return v.f[d] if d in v.f else 0

def setitem(v,d,val):
    "Set the element of v with label d to be val"
    assert d in v.D
    v.f[d] = val# if val != 0 else pass

def equal(u,v):
    "Returns true iff u is equal to v"
    assert u.D == v.D
    for i in u.D:
        if u[i] != v[i]:
            return False
    return True
    #return u.f == v.f

def add(u,v):
    "Returns the sum of the two vectors"
    assert u.D == v.D
    return Vec(u.D, {d: u[d] + v[d] for d in u.D})
    
def dot(u,v):
    "Returns the dot product of the two vectors"
    assert u.D == v.D
    return sum([ u[d] * v[d] for d in u.D ])

def scalar_mul(v, alpha):
    "Returns the scalar-vector product alpha times v"  
    return Vec(v.D, {d: v[d] * alpha for d in v.D})
    

def neg(v):
    "Returns the negation of a vector"
    return scalar_mul(v, -1)

##### NO NEED TO MODIFY BELOW HERE #####
class Vec:
    """
    A vector has two fields:
    D - the domain (a set)
    f - a dictionary mapping (some) domain elements to field elements
        elements of D not appearing in f are implicitly mapped to zero
    """
    def __init__(self, labels, function):
        self.D = labels
        self.f = function

    __getitem__ = getitem
    __setitem__ = setitem
    __neg__ = neg
    __rmul__ = scalar_mul #if left arg of * is primitive, assume it's a scalar

    def __mul__(self,other):
        #If other is a vector, returns the dot product of self and other
        if isinstance(other, Vec):
            return dot(self,other)
        else:
            return NotImplemented  #  Will cause other.__rmul__(self) to be invoked

    def __truediv__(self,other):  # Scalar division
        return (1/other)*self

    __add__ = add

    def __radd__(self, other):
        "Hack to allow sum(...) to work with vectors"
        if other == 0:
            return self
    
    def __sub__(a,b):
         "Returns a vector which is the difference of a and b."
         return a+(-b)

    __eq__ = equal

    def __str__(v):
        "pretty-printing"
        try:
            D_list = sorted(v.D)
        except TypeError:
            D_list = sorted(v.D, key=hash)
        numdec = 3
        wd = dict([(k,(1+max(len(str(k)), len('{0:.{1}G}'.format(v[k], numdec))))) if isinstance(v[k], int) or isinstance(v[k], float) else (k,(1+max(len(str(k)), len(str(v[k]))))) for k in D_list])
        # w = 1+max([len(str(k)) for k in D_list]+[len('{0:.{1}G}'.format(value,numdec)) for value in v.f.values()])
        s1 = ''.join(['{0:>{1}}'.format(k,wd[k]) for k in D_list])
        s2 = ''.join(['{0:>{1}.{2}G}'.format(v[k],wd[k],numdec) if isinstance(v[k], int) or isinstance(v[k], float) else '{0:>{1}}'.format(v[k], wd[k]) for k in D_list])
        return "\n" + s1 + "\n" + '-'*sum(wd.values()) +"\n" + s2

    def __repr__(self):
        return "Vec(" + str(self.D) + "," + str(self.f) + ")"

    def copy(self):
        "Don't make a new copy of the domain D"
        return Vec(self.D, self.f.copy())

## Vec_Util Implementation

In [ ]:
def list2vec(L):
    """Given a list L of field elements, return a Vec with domain {0...len(L)-1}
    whose entry i is L[i]

    >>> list2vec([10, 20, 30])
    Vec({0, 1, 2},{0: 10, 1: 20, 2: 30})
    """
    return Vec(set(range(len(L))), {k: val for (k, val) in enumerate(L)} )

def zero_vec(D):
    """Returns a zero vector with the given domain
    """
    return Vec(D, {})


In [ ]:
v = Vec({"A", "B", "C"}, {"A": 1})
print(v)

In [ ]:
print(repr(v))

### List to Vec Procedure

In [ ]:
def list2vec(L):
    return Vec(set(range(len(L))), {k: val for (k, val) in enumerate(L)} )

## Solving a Triangular System of Linear Equations
As a step towards Computational Problem Solving, we describe an algorithm for solving a system if the system has a special form.
### Upper-Triangular Systems
A *upper-triangular system of linear equations* has the form:

$$
\begin{align}
    \begin{bmatrix} 
        a_11 & a_12 & a_13 & \dots & a_{1,n-1} & a_{1,n}
    \end{bmatrix}  * x = \beta_1 \\
    \begin{bmatrix} 
        0 & a_22 & a_23 & \dots & a_{2,n-1} & a_{2,n}
    \end{bmatrix} * x = \beta_2 \\
    \begin{bmatrix} 
        0 & 0 & a_33 & \dots & a_{2,n-1} & a_{2,n}
    \end{bmatrix} * x = \beta_3 \\
    \begin{bmatrix} 
        0 & 0 & 0 & \dots & a_{n-1,n-1} & a_{n-1,n}
    \end{bmatrix} * x = \beta_4 \\
    \begin{bmatrix} 
        0 & 0 & 0 & \dots & 0 & a_{n,n}
    \end{bmatrix} * x = \beta_5 \\
\end{align}
$$

That is,

- The first vector need not have any zeroes,
- The second vector has a zero in the first position,
- The third vector has a zero in the first and second positions,
- The fourth vector has zeroes in the first, second, and their position,

- $ \vdots $

- The $n - 1^{st}$ vector is all zeroes except possibly for the $n - 1^{st}$ and $n^{th}$ entries
- The $n^{th}$ vector is all zeroes except possibly for the $n^{th}$ entry.

#### Example Using 4-Vector
$$
\begin{align}
    \begin{bmatrix}
    1 & 0.5 & -1 & 4
    \end{bmatrix} * x = -8 \\
    \begin{bmatrix}
    0 & 3 & 3 & 2
    \end{bmatrix} * x = 3 \\
    \begin{bmatrix}
    0 & 0 & 1 & 5
    \end{bmatrix} * x = -4 \\
    \begin{bmatrix}
    0 & 0 & 0 & 2
    \end{bmatrix} * x = 6 \\
\end{align}
$$

The origin of the term *upper_triangular system* should be apparent by considering the positions of the nonzero entries: they form a triangle.

Writing $x = \begin{bmatrix} x_1 & x_2 & x_3 & x_4 \end{bmatrix}$ and using the definition of dot-product, we can rewrite this system as four ordinary equations inthe (scalar) unknowns $x_1,x_2,x_3,x_4$:

$
\begin{align}
1_{x_1} + 0.5_{x_2} - 2_{x_3} + 4_{x_4} = -8 \\
         3_{x_2} - 3_{x_3} + 2_{x_4} = 3 \\
                      1_{x_3} + 5_{x_4} = -4 \\
                                2_{x_4} = 5 \\
\end{align}
$

### Backward Substitution
This suggest a solution stategy. First, solve for $x_3$ using the foruth equation. Plug the resulting value for $x_4$ into the third equations, and solve for $x_3$. Plug the values for $x_3$ and $x_4$ into the second equation and solve for $x_2$. Plug the values for $x_2$, $x_3$, and $x_4$ into the first aequation and solve for $x_1$. In ech iteration, only one variable needs to be solve for. Thus the above system is solve.
#### Implementation

In [3]:
def triangular_solve(rowlist, label_list, b):
    D = rowlist[0].D
    x = zero_vec(D)
    for i in reversed(range(len(D))):
        c = label_list[i]
        row = rowlist[i]
        x[c] = (b[i] - row * x) / row[c]
    return x

## Linear Combinations
### From coefficient to linear combination
There are two related computational problems, the forward problem (given an element of the domain, find the image under the function) and the backward problem (given an element of the co-domain, find any pre-image if there is one).

Solving the forward problem is easy:

In [5]:
def lin_comb(vlist, clist):
    """
        input: a list vlist of vectors, a list clist of the same length consisting of scalars
        output: the vector that is the linear combination of vectors in vlist with corresponding
        coefficients clist
    """
    return sum([v * scalar for (v, scalar) in zip(vlist, clist)])

### From linear combination to coefficients
The first question is: can you solve the backward problem? that is, can you obtain a pre-image of b under f? the second question is: how can we tell whether ther is a single solution? If there are multiple pre-images of b, we cannot be confident that we have calculated the true number of garden gnomes.

The first question is a computational problem:

*Expressing a given vector as a linear combination of other given vectors*
- input: a vector $b$ and a list $[v_1,\dots, v_n]$ of $n$ vectors
- output: a list $[a_1, \dots, a_n]$ of coefficients such that: 
$$
b = \alpha_1v_1 + \dots + \alpha_n v_n
$$

or a report that none exists.

Finding a linear combination of given vectors $v_1, \dots, v_n$ that equals a given vector $b$ is equivalent to solving a linear system.